# Find the right OMI file to visualize

Files have two date/times in their names:
- 1st = acquisition
- 2nd = when processed

They are 'filed' based on the processing date.

Sometimes the same orbit data is processed twice, and we recieve both processed files, in that case the most recent one should be used.

`omi
└── archive
    ├── 2020-04-01
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t1653-o83566_v003-2020m0401t114126.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t1832-o83567_v003-2020m0401t114119.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2011-o83568_v003-2020m0401t114113.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2150-o83569_v003-2020m0401t114112.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0331t2328-o83570_v003-2020m0401t184121.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t0107-o83571_v003-2020m0401t184102.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t0246-o83572_v003-2020m0401t185719.he5
    │   └── OMI-Aura_L2-OMSO2_2020m0401t0425-o83573_v003-2020m0401t185732.he5
    ├── 2020-04-02
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t1736-o83581_v003-2020m0402t120605.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t1915-o83582_v003-2020m0402t120601.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t2054-o83583_v003-2020m0402t120552.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0401t2233-o83584_v003-2020m0402t120550.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0012-o83585_v003-2020m0402t143344.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0151-o83586_v003-2020m0402t143855.he5
    │   ├── OMI-Aura_L2-OMSO2_2020m0402t0330-o83587_v003-2020m0402t143855.he5
    │   └── OMI-Aura_L2-OMSO2_2020m0402t0508-o83588_v003-2020m0402t203226.he5
    ├── 2020-04-03`
    
The satellite passes overhead the Equator at approximately 1:45 pm, local time (I assume this is ignoring Daylight Saving). This can be used to determine which file, each of which contains a single data swath, is the right one for a particular area. For example, the file suitable for PNG-Solomons is acquired at about 0245 UTC.

Data for a particular UTC date can be found in the folder named by that date (typically up to about 0500 UTC), and in the following folder (typically from about 1600 UTC).

WHEN DO THESE FILES ARRIVE FORM NASA?
    

In [52]:
import os
import datetime
from datetime import timedelta
import pytz
import numpy as np

## Example

In [ ]:
datadir = '/home/sherburn/Work/GeoNet/omi/archive'

area = 'vanuatu'

date = '2020-04-04'


In [57]:
def prevday(date):
    #find previous date given a date string
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    prev = (dt+ timedelta(days=-1)).date()
    return prev.strftime('%Y-%m-%d')

In [18]:
def passtime(area, date):
    #find the theoretical pass time for the OMI staellite at the lititude of the area provided
    #satellite nominally passes at 1:45 pm
    
    #timezones
    zones = {'pngsol':'Pacific/Bougainville', 'vanuatu':'Pacific/Bougainville', 'sft':'Pacific/Tongatapu', 'nz':'NZ', 'kerm':'NZ' }
    
    obstime = '13:45:00' # 1:45 pm local time
    dt = date+' '+obstime
    dt = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
    
    print ('for ', area)
    timezone = pytz.timezone(zones[area])
    dt = timezone.localize(dt)
    dtutc = dt.astimezone(pytz.utc)
    print ('passtime = ', dtutc)
    return (dtutc)

In [54]:
def nearest_file(files, passutc):
    #find nearest file to satellite pass time
    #file name example OMI-Aura_L2-OMSO2_2020m0404t2125-o83627_v003-2020m0405t114954.he5
    print (passutc)
    diffmin = 1e9
    for file in files:
        print (file)
        swathdate = file[18:22]+'-'+file[23:25]+'-'+file[25:27]
#         print (swathdate)
        swathtime = file[28:30]+':'+file[30:32]
#         print (swathtime)
        swathdt = swathdate+' '+swathtime
        swathdt = datetime.datetime.strptime(swathdt, '%Y-%m-%d %H:%M')
        timezone = pytz.timezone('UTC')
        swathdt = timezone.localize(swathdt)
#         print (swathdt)
        tdiff = np.abs((swathdt - passutc).total_seconds())
        print (tdiff)
        if tdiff < diffmin:
            diffmin = tdiff
            filemin = file
            print ('new filemin = ', filemin)
            print ('new diffmin = ', diffmin)
        print ('-----')

In [38]:
f = 'OMI-Aura_L2-OMSO2_2020m0404t2125-o83627_v003-2020m0405t114954.he5'
swathdate = f[18:22]+'-'+f[23:25]+'-'+f[25:27]
print (swathdate)
swathtime = f[28:30]+':'+f[30:32]
print (swathtime)

swathdt = datetime.datetime.strptime(swathdate+' '+swathtime, '%Y-%m-%d %H:%M')
swathdt

2020-04-04
21:25


datetime.datetime(2020, 4, 4, 21, 25)

In [19]:
passutc = passtime('vanuatu', date)

for  vanuatu
passtime =  2020-04-04 02:45:00+00:00


In [20]:
print (passutc)

2020-04-04 02:45:00+00:00


In [58]:
prevdate = prevday(date)
prevdate

'2020-04-03'

In [67]:
#list files in current and previous day folders
files = []
for d in [prevdate, date]:
    print (d)
    dir = os.path.join(datadir, d)
    dirfiles = [os.path.join(dir, file) for file in os.listdir(dir)]
    for file in dirfiles:
        files.append(file)
    

2020-04-03
2020-04-04


In [68]:
files

['/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0402t2316-o83599_v003-2020m0403t154214.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0402t2137-o83598_v003-2020m0403t120201.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0403t0234-o83601_v003-2020m0403t153703.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0402t1959-o83597_v003-2020m0403t115802.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0403t0413-o83602_v003-2020m0403t154735.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0403t0055-o83600_v003-2020m0403t154228.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0402t1820-o83596_v003-2020m0403t120201.he5',
 '/home/sherburn/Work/GeoNet/omi/archive/2020-04-04/OMI-Aura_L2-OMSO2_2020m0403t1903-o83611_v003-2020m0404t122543.he5',
 '/home/sherburn/Work/GeoNet/omi/archive

In [69]:
for file in files:
    nearest_file(files, passutc)

2020-04-04 02:45:00+00:00
/home/sherburn/Work/GeoNet/omi/archive/2020-04-03/OMI-Aura_L2-OMSO2_2020m0402t2316-o83599_v003-2020m0403t154214.he5


ValueError: time data 'k/Ge-Ne-t/ mi:/a' does not match format '%Y-%m-%d %H:%M'